In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.chdir('/content/drive/MyDrive/sharif/DeepLearning/ipython(guide)')

In [4]:
import numpy as np
import codecs
import os
import random
import pandas
from keras import backend as K
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Conv1D, Dense, Lambda, Permute, Dropout , Softmax
from keras.layers import Conv2D, MaxPooling1D , Flatten , BatchNormalization
from keras.optimizers import SGD
import ast
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import gensim
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [5]:
import re
from sklearn.preprocessing import MultiLabelBinarizer

limit_number = 750
data = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv',index_col=0,converters={'body': eval})
data = data.dropna().reset_index(drop=True)
X = data["body"].values.tolist()
y = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv')
labels = []
tag=[]
for item in y['tag']:
  labels += [i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' ']
  tag.append([i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' '])
labels = list(set(labels))
mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(tag)

In [6]:
len(labels)

78

In [7]:
sentence_maxlen = max(map(len, (d for d in X)))
print('sentence maxlen', sentence_maxlen)

sentence maxlen 300


In [8]:
freq_dist = pandas.read_csv('../Data/FreqDist_sorted.csv',index_col=False)
vocab=[]
for item in freq_dist["word"]:
  try:
    word=re.sub(r"[\u200c-\u200f]","",item.replace(" ",""))
    vocab.append(word)
  except:
    pass
  
print(vocab[10])

زبان


In [9]:
vocab = sorted(vocab)
vocab_size = len(vocab)

In [10]:
print('vocab size', len(vocab))
w2i = {w:i for i,w in enumerate(vocab)}
# i2w = {i:w for i,w in enumerate(vocab)}
print(w2i["زبان"])

vocab size 225345
129280


In [11]:
def vectorize(data, sentence_maxlen, w2i):
    vec_data = []
    
    for d in data:
       
        vec = [w2i[w] for w in d if w in w2i]
        pad_len = max(0, sentence_maxlen - len(vec))
        vec += [0] * pad_len
        vec_data.append(vec)
        # print(d)
        
    vec_data = np.array(vec_data)
    
    return vec_data

vecX = vectorize(X, sentence_maxlen, w2i)
vecY=Y

In [12]:
X_train, X_test, y_train, y_test = train_test_split(vecX, vecY, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
print('train: ', X_train.shape , '\ntest: ', X_test.shape , '\nval: ', X_val.shape ,"\ny_tain:",y_train.shape )
# print(vecX[0])

train:  (12935, 300) 
test:  (4312, 300) 
val:  (4312, 300) 
y_tain: (12935, 78)


In [13]:
embd_dim = 300

In [14]:
# embed_model = gensim.models.Word2Vec(X, size=embd_dim, window=5, min_count=5)
# embed_model.save('word2vec_model')

In [15]:
embed_model=gensim.models.Word2Vec.load('word2vec_model')

word2vec_embd_w = np.zeros((vocab_size, embd_dim))
for word, i in w2i.items():
  if word in embed_model.wv.vocab:
        embedding_vector =embed_model[word]
  
        # words not found in embedding index will be all-zeros.
        word2vec_embd_w[i] = embedding_vector
        

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [ ]:
def Net(vocab_size, embd_size, sentence_maxlen, glove_embd_w):
    sentence = Input((sentence_maxlen,), name='SentenceInput')
    
    # embedding
    embd_layer = Embedding(input_dim=vocab_size, 
                           output_dim=embd_size, 
                           weights=[word2vec_embd_w], 
                           trainable=False,
                           name='shared_embd')
    embd_sentence = embd_layer(sentence)
    embd_sentence = Permute((2,1))(embd_sentence)
    embd_sentence = Lambda(lambda x: K.expand_dims(x, -1))(embd_sentence)
    
    # cnn

    cnn_1 = Conv2D(1,name='cnn_1', 
                 kernel_size=(5, sentence_maxlen),
                 activation='relu')(embd_sentence)

    cnn_2 =  Lambda(lambda x: K.sum(x, axis=3),name='cnn_2')(cnn_1)
    max_pool_1 = MaxPooling1D(3,name='max_pool_1')(cnn_2)
    # bn_1 = BatchNormalization(name='bn_1')(max_pool_1)
    cnn_3 = Lambda(lambda x: K.sum(x, axis=2),name='cnn_3')(max_pool_1)
    # bn_2 = BatchNormalization(name='bn_2')(cnn_3)

    flatten = Flatten(name='flatten')(cnn_3)
    dense_1 = Dense(400,activation="relu",name='dense_1')(flatten)
    dense_2 = Dense(300,activation="relu",name='dense_2')(dense_1)
    dense_3 = Dense(200,activation="relu",name='dense_3')(dense_2)
    dense_4 = Dense(len(labels),activation="sigmoid",name='dense_4')(dense_3)
    out = Softmax(name='Soft_max_out')(dense_4)
    
    

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = Model(inputs=sentence, outputs=out, name='sentence_claccification')
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=["accuracy", "binary_accuracy",
    "categorical_accuracy",]) 
    return model


model = Net(vocab_size, embd_dim, sentence_maxlen,word2vec_embd_w)
print(model.summary())


Model: "sentence_claccification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SentenceInput (InputLayer)   [(None, 300)]             0         
_________________________________________________________________
shared_embd (Embedding)      (None, 300, 300)          67603500  
_________________________________________________________________
permute (Permute)            (None, 300, 300)          0         
_________________________________________________________________
lambda (Lambda)              (None, 300, 300, 1)       0         
_________________________________________________________________
cnn_1 (Conv2D)               (None, 296, 1, 1)         1501      
_________________________________________________________________
cnn_2 (Lambda)               (None, 296, 1)            0         
_________________________________________________________________
max_pool_1 (MaxPooling1D)    (None, 98, 1) 

In [ ]:
# model.fit(X_train, y_train,
#             batch_size=32,
#             epochs=5,
#             validation_data=(X_val, y_val))

In [ ]:
# model = Net(vocab_size, embd_dim, sentence_maxlen,word2vec_embd_w)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50) # Model stop training after 50 epoch where validation loss didnt decrease
mc = ModelCheckpoint('best_cnn_4fc.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True) #You save model weight at the epoch where validation loss is minimal
model.fit(X_train, y_train, batch_size=32,epochs=20,verbose=1,validation_data=(X_val, y_val),callbacks=[es,mc])#you can run for 1000 epoch btw model will stop after 50 epoch without better validation loss


Epoch 1/20
405/405 [==============================] - 47s 113ms/step - loss: 0.1160 - accuracy: 0.0176 - binary_accuracy: 0.9763 - categorical_accuracy: 0.0176 - val_loss: 0.1168 - val_accuracy: 0.0183 - val_binary_accuracy: 0.9761 - val_categorical_accuracy: 0.0183

Epoch 00001: val_loss improved from inf to 0.11681, saving model to best_cnn_4fc.h5
Epoch 2/20
405/405 [==============================] - 47s 116ms/step - loss: 0.1130 - accuracy: 0.0211 - binary_accuracy: 0.9770 - categorical_accuracy: 0.0211 - val_loss: 0.1167 - val_accuracy: 0.0230 - val_binary_accuracy: 0.9761 - val_categorical_accuracy: 0.0230

Epoch 00002: val_loss improved from 0.11681 to 0.11671, saving model to best_cnn_4fc.h5
Epoch 3/20
405/405 [==============================] - 47s 116ms/step - loss: 0.1137 - accuracy: 0.0244 - binary_accuracy: 0.9768 - categorical_accuracy: 0.0244 - val_loss: 0.1166 - val_accuracy: 0.0260 - val_binary_accuracy: 0.9761 - val_categorical_accuracy: 0.0260

Epoch 00003: val_loss im

In [16]:
# model.save('CNN_2_with_binary.h5')

from keras.models import load_model
model = load_model('CNN_2_with_binary.h5')

Evaluation

In [17]:
pred=model.predict(X_test)
# For evaluation: If the probability > 0.5 you can say that it belong to the class.

In [18]:
print(pred[0])#example

[0.01505763 0.0183046  0.02220036 0.0082808  0.00946112 0.00824184
 0.0191835  0.00858157 0.00843826 0.01414727 0.02134026 0.01801053
 0.01779321 0.0083077  0.00830454 0.02202772 0.01548668 0.01007422
 0.00937071 0.01050385 0.01235036 0.01693296 0.00836943 0.01972198
 0.01228632 0.0156456  0.01703042 0.01333099 0.0156227  0.01250619
 0.00989637 0.01291912 0.01114158 0.02152479 0.00915246 0.02188545
 0.01906556 0.00947602 0.00898158 0.00870321 0.01314218 0.01184288
 0.01173529 0.01828727 0.01816805 0.00824842 0.0084116  0.01559324
 0.01215253 0.00891273 0.00831708 0.0082748  0.00826448 0.01309263
 0.01692103 0.00821617 0.00889599 0.00826062 0.01160371 0.02015395
 0.01070112 0.00822996 0.02007708 0.01274718 0.01028533 0.01322484
 0.00837617 0.01163152 0.00946885 0.01662587 0.01313476 0.01044647
 0.01346069 0.0088749  0.00834925 0.01534267 0.0114846  0.00938456]


In [33]:
np.mean(pred[0]) + np.sqrt(np.var(pred[0])) 

0.017101556

In [36]:
y_pred=[]
for l in pred:
  temp=[]
  for value in l:
    if value >= np.mean(pred[0]) + .75*np.sqrt(np.var(pred[0])) :
      temp.append(1)
    else:
      temp.append(0)
  y_pred.append(temp)

In [37]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.03      0.59      0.06       130
           1       0.04      0.42      0.07       165
           2       0.03      1.00      0.06       143
           3       0.00      0.00      0.00        19
           4       0.47      0.90      0.62        78
           5       0.00      0.00      0.00        18
           6       0.11      0.69      0.18       165
           7       0.00      0.00      0.00        43
           8       0.00      0.00      0.00        27
           9       0.05      0.34      0.08        88
          10       0.03      0.99      0.06       144
          11       0.13      0.71      0.22        82
          12       0.30      0.96      0.46       121
          13       0.00      0.00      0.00        19
          14       0.00      0.00      0.00        13
          15       0.04      1.00      0.08       175
          16       0.08      0.84      0.15       133
          17       0.10    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
